# **Introducción a Python para Análisis de Datos**
## Capítulo 4: Limpieza de Datos - Soluciones a Ejercicios
---
Acerca de este notebook
* **Autor:** Juan Martin Bellido
* **Descripción:** *este notebook incluye las soluciones a los ejercicios del capítulo 4*
* **¿Feedback? ¿comentarios?** Por favor compártelo conmigo escribiéndome por [LinkedIn](https://www.linkedin.com/in/jmartinbellido/)


# Capítulo 4: Ejercicios
---

### Ejercicio #1

##### EX 1.A Identificar columnas con valores nulos en el dataframe.
##### EX 1.B ¿Cuántas películas hay con valores nulos para el campo *director_name*?
##### EX 1.C Reemplazar valores nulos en *director_name* por "unknown".

> Dataset https://data-wizards.s3.amazonaws.com/datasets/movies.csv


In [ ]:
import pandas as pd
df_movies = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/movies.csv")
df_movies.dtypes

color                         object
director_name                 object
num_critic_for_reviews       float64
duration                     float64
director_facebook_likes      float64
actor_3_facebook_likes       float64
actor_2_name                  object
actor_1_facebook_likes       float64
gross                        float64
genres                        object
actor_1_name                  object
movie_title                   object
num_voted_users                int64
cast_total_facebook_likes      int64
actor_3_name                  object
facenumber_in_poster         float64
plot_keywords                 object
movie_imdb_link               object
num_user_for_reviews         float64
language                      object
country                       object
content_rating                object
budget                       float64
title_year                   float64
actor_2_facebook_likes       float64
imdb_score                   float64
aspect_ratio                 float64
m

In [ ]:
# EX 1.A
df_movies.isna().any()

color                         True
director_name                 True
num_critic_for_reviews        True
duration                      True
director_facebook_likes       True
actor_3_facebook_likes        True
actor_2_name                  True
actor_1_facebook_likes        True
gross                         True
genres                       False
actor_1_name                  True
movie_title                  False
num_voted_users              False
cast_total_facebook_likes    False
actor_3_name                  True
facenumber_in_poster          True
plot_keywords                 True
movie_imdb_link              False
num_user_for_reviews          True
language                      True
country                       True
content_rating                True
budget                        True
title_year                    True
actor_2_facebook_likes        True
imdb_score                   False
aspect_ratio                  True
movie_facebook_likes         False
dtype: bool

In [ ]:
# EX 1.B
cond = df_movies['director_name'].isna()
len(df_movies[cond])

102

In [ ]:
# EX 1.C
df_movies['director_name'] = df_movies['director_name'].fillna('unknown')

### Ejercicio #2

Detectar empresas que sean outliers en cantidad de empleados.

##### EX 2.A Utilizando distribución normal
##### EX 2.B Utilizando el método del rango intercuartílico

> Dataset https://data-wizards.s3.amazonaws.com/datasets/fortune1000.csv

In [ ]:
import pandas as pd
from scipy import stats
df_fortune = pd.read_csv('https://data-wizards.s3.amazonaws.com/datasets/fortune1000.csv')
df_fortune.dtypes

Rank          int64
Company      object
Sector       object
Industry     object
Location     object
Revenue       int64
Profits       int64
Employees     int64
dtype: object

In [ ]:
# 2.A
# normalizamos variable
df_fortune["z_Employees"] = stats.zscore(df_fortune["Employees"],nan_policy='omit').round(2)

In [ ]:
# filtramos filas, seleccionamos columnas, ordenamos filas
cond = abs(df_fortune['z_Employees'])>3
df_fortune[cond][['Company','Employees','z_Employees']].sort_values("z_Employees",ascending=False)

,Company,Employees,z_Employees
0,Walmart,2300000,25.08
217,Yum Brands,505000,5.22
16,Kroger,431000,4.40
108,McDonald’s,420000,4.28
30,IBM,411798,4.19
27,Home Depot,385000,3.89
37,Target,341000,3.41
47,UPS,341240,3.41
10,General Electric,333000,3.32
3,Berkshire Hathaway,331000,3.30


In [ ]:
# 2.B
# calculamos el IQR
IQR = df_fortune["Employees"].quantile(0.75) - df_fortune["Employees"].quantile(0.25)

In [ ]:
# calculamos los límites superiores e inferiores, según el método IQR
sup_lim = IQR * 1.5 + df_fortune["Employees"].quantile(0.75)
inf_lim = df_fortune["Employees"].quantile(0.25) - IQR * 1.5

In [ ]:
# creamos condiciones para capturar las filas donde el campo empleados quede por fuera de los límites
cond = df_fortune["Employees"] > sup_lim
cond_2 = df_fortune["Employees"] < inf_lim

# filtramos, seleccionamos columnas, ordenamos filas
df_fortune[cond | cond_2][['Company','Employees']].sort_values("Employees",ascending=False)

,Company,Employees
0,Walmart,2300000
217,Yum Brands,505000
16,Kroger,431000
108,McDonald’s,420000
30,IBM,411798
...,...,...
239,Toys “R” Us,62000
35,Alphabet,61814
435,Jones Lang LaSalle,61500
13,Chevron,61500


### Ejercicio #3

El dataset contiene productos de Amazon. Limpiar las categorías de productos hasta obtener las 3 primeras para cada producto.

> Dataset https://data-wizards.s3.amazonaws.com/datasets/amzn_products.txt



In [ ]:
import pandas as pd
df_amzn_products = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/amzn_products.txt",sep="\t")
df_amzn_products.dtypes

asin          object
brand         object
categories    object
dtype: object

In [ ]:
# utilizamos método .str.split() para separar valores de un campo de texto según delimitador
df_amzn_products['categories'].str.split(pat=',',expand=True,n=3)\
    .rename(columns={0:'cat_1',1:'cat_2',2:'cat_3'})\
    .join(df_amzn_products)\
    .loc[:,['asin','brand','cat_1','cat_2','cat_3']]

,asin,brand,cat_1,cat_2,cat_3
0,B0168YIWSI,Microsoft,Electronics,Computers,Computer Accessories
1,"B018K251JE,B00VILQKQ8",Boytone,Electronics,Portable Audio & Video,Home Audio & Theater
2,B00C78VIUE,Sanus,Audio & Video Accessories,TV Mounts,TV Accessories & Parts
3,B075WKS4D8,Ultimate Ears,Portable Bluetooth Speakers,Bluetooth & Wireless Speakers,Portable Audio & Video
4,B000YA33DC,Lowepro,Photography,Photography Bags Cases,Camera Bags
5,B00505EZYW,Corsair,Computers,Memory (RAM),Computers & Accessories
6,B00WBS31OQ,"Sdi Technologies, Inc.",Portable Bluetooth Speakers,Stereos,Computers
7,"B0172DS3ME,B00NTJKAXG",Verizon Wireless,Computers,Networking,Home Networking & Connectivity
8,B00R5WGT2E,JVC,Electronics,Portable Audio,Auto & Tires
9,B00V631478,JBL,Electronics,Car & Vehicle Electronics,Car Electronics
